# Section 12 - Deep Learning - 11/11/22

Deep Learning is the application of neural networks to problems in machine learning. There are a wide variety of popular deep learning architecures that are catered to different problems and domains such as computer vision, signal processing, natural language, and many more. In this section we will discuss the basics of deep learning and some interesting applications of deep learning.

## What is a Neural Network

As mentioned before, deep learning is the application of neural networks to solve problems in machine learning. This raises the question "What is a neural network?" A neural network is a learning framework originally designed to construct a learning process modeled after how the human brain sends signals. A neural network contains individual units called "neurons" that are connected to one another in a graph like structure. Each neuron gets inputs from some other neurons and based on the signal of inputs makes a decision on how it should output its own signal to all of the neurons it serves as input to. Typically we organize neurons into layers where each layer gets input from every neuron in the previous layer and sends output to each node in the next layer. We call the first and last layers the input and output layers respectively and refer to intermediate layers as "hidden layers".

<h1><center>


![NN](Neural_Network.png)
  
</center></h1>

We tend to represent how a neuron interacts with its neighbors using a matrix of weights that denotes how much it weighs the signal of each input neuron. We also tend to apply an "activation function" that determines what signal the neuron should output given each of its input signals. For a given neuron $i$ with input signals $x_1,\dots,x_n$ we usually express this as follows. Here $o_i$ denotes the intermediate result, $a_i$ denotes the "activation" or output signal, and $\sigma()$ denotes the activation function.

$$ o_i = \sum_{j=1}^{n} w_{ij} x_j + b_i $$

$$ a_i = \sigma(o_i)$$


## How Does a Neural Network Learn?

The next question is how neural networks are able to learn. The key to training these models is a loss function - a function we apply to our model predictions to measure how well the model did at predicting the samples we gave it. Typically if we have $n$ samples $x_1,\dots,x_n$ with correspinding labels or outputs $y_1,\dots,y_n$ we can compute the total loss as follows with a loss function $l(x,y;w)$. Let $h(x_i)$ denote the models prediction for sample $x_i$.

$$ Loss = \frac{1}{n}\sum_{i=1}^{n} l(h(x_i),y_i) $$

An example loss function is the linear regression loss:

$$l(h(x),y) = (h(x) - y)^2$$


As mentioned before the network is parameterized by weights $W$ and biases $b$ at each layer. Using this loss function we can compute the gradient of each of these parameters with respect to the final loss. This gives us a measure of the sensitivity of the loss with respect to each of the model weights. Recall from calculus that the gradient also tells us the direction of steepest ascent. Using the gradients we can make slight updates to our weights that reduce the loss, leading to our model better fitting the training data. This update typically looks something like the following where $\alpha$ denotes the "step-size" of our update:

$$w_{t+1} = w_t - \alpha\nabla_w l$$

This update method is known as gradient descent and has many variations used throughout machine learning. For more on optimization methods consider reading about Newton's Method, Stochastic Gradient Descent, and ADAM optimization.


## Practice Problem 1

Lets assume we have a simple neural network as shown above with 2 input neuron, 3 hidden neurons, and 1 output neuron. How many weights and biases do we need for the first layer? How many for the second layer?





Now lets consider we have the following weight matrices:

$$W_1 = \begin{bmatrix} 1 & 0 \\ -2 & -1 \\ 1 & -2 \end{bmatrix}, b_1 = \begin{bmatrix} 1\\ 0 \\ 0 \end{bmatrix}, W_2 = \begin{bmatrix} 1 & -1 & 1\end{bmatrix}, b_2 = -1 $$

Also consider after the hidden layer we apply the rectified linear activation function $\sigma(x) = ReLU(x) = max(0,x)$. Asssume that we apply no activation to the output layer. This gives us the following series of operations:

$$ o_1 = W_1 x + b_1 $$
$$a_1 = ReLU(o_1)$$
$$o_2 = W_2a_1 + b_2$$

Lets compute the model output on the data point $x = \begin{bmatrix} 1 \\ -1 \end{bmatrix}$ with target $y = 8$:

$$o_1 = \begin{bmatrix} 2 \\ -1 \\ 3 \end{bmatrix}$$

$$a_1 = \begin{bmatrix} 2 \\ 0 \\ 3 \end{bmatrix}$$

$$o_2 = 4$$

Thus we get the model prediction $h(x) =  4$. Now lets use the linear regression loss we saw earlier to compute the loss of this prediction:

$$l(4,8) = (4 - 8)^2 = 16$$

We have completed one complete forward pass computation of this simple network on our training point. For the sake of time we will avoid going through computing the model gradients and updating parameters.


Next lets implement this simple two-layer model into a templated class. For this class we initialize each of the weights with 


In [1]:
import numpy as np

class smallForwardNetwork:

    def __init__(self):
        # Initialize weights and biases
        self.W1 = np.array([[1,0],[-2,-1],[1,-2]])
        self.W2 = np.array([1,-1,1]).reshape(1,-1)
        self.b1 = np.array([1,0,0]).reshape(-1,1)
        self.b2 = -1
        return

    def forward(self, x):

        o1 = self.W1 @ x + self.b1
        a1 = np.maximum(o1,np.zeros(o1.shape))
        output = self.W2 @ a1 + self.b2
        return output


    def compute_loss(self,prediction,target):

        loss = np.square(prediction - target)
        return loss

 


Now lets test this model against our hand-written solution:

In [2]:
model = smallForwardNetwork()
x = np.array([1,-1]).reshape(-1,1)
y = 8.0

prediction = model.forward(x)
loss = model.compute_loss(prediction,y)
print(f"Our prediction on x: {prediction}")
print(f"Our model loss on x: {loss}")

Our prediction on x: [[4.]]
Our model loss on x: [[16.]]


# Applications

Deep learning has very interesting applications in just about any field you could imagine. In genetics in particular deep learning has been used to identify genetic factors that may cause common medical disorders. Genetics problems tend to inherently rely on sequential data, where input data is stored in a specific order that plays an important role in the "meaning" of the data. Another popular area of research where sequential data is very common is natural language processing, where word order and sentence structure determine how a sentence should be interpreted. In general settings like we outlined before we might have an input $x$ where each entry $x_1,\dots,x_n$ is an independent feature we'd like to use for prediction. However in sequential models we want to capture the sequential structure and thus we want to predict $x_i$ given some structural information. In the most basic setting we could predict $x_i$ given some previous information $x_1,\dots,x_{i-1}$. This is the basic inspiration for the Recurrent Neural Network.

## Recurrent Neural Networks(RNNs)

A recurrent neural network is similar to a standard feed-forward network like we saw above, except it aims to also track historical information through a "hidden state" or "hidden vector". As an example lets consider the NLP task of labeling the parts of speech for each word in a sentence. In an RNN our hidden layer takes in two inputs at each timestep. At time $t$ we pass the hidden layer the $t$th word $x_t$ and we also pass the current hidden state $h_{t-1}$, which is a representation of the "meaning" of words $x_1$ through $x_{t-1}$. Then the hidden layer outputs a predicted part-of-speech tag $y_t$ (for example "NOUN"), as well as an updated hidden state $h_t$ that now includes $x_1$ through $x_t$, which we will pass in at the next time step. 


<h1><center>

![RNN](RNN.png)

</center></h1>


This architecture has proven successfull over traditional feed-forward networks for sequence prediction tasks. However the model has computational limitations since we have to predict each element in the sequence one at a time. In practice RNNs have begun to be replaced with another more complicated class of models called "Transformers". We won't go into transformers in this section but if you would like to read more the paper on transformers can be found at https://arxiv.org/abs/1706.03762.










## Generative Adversarial Networks (GANs)

An area of deep learning I'm particularly interested in is generative models. Unlike discriminative models where we are trying to directly make predictions on our training data, generative models instead aim to capture the distribution of the training samples in order to generate new samples from that distribution. This is a particularly popular method in areas like text-to-speech as well as image and video generation, but even has applications extending into biology and chemistry. One interesting architecture is the Generative Adversarial Network.


GANs are a type of generative deep learning model that simultaneously learns two neural networks - a generator and a discriminator. The generator's job is to generate synthetic samples that are sampled from the training domain. The discriminator's job is to learn to distinguish between real samples and the fake samples generated by the generator. This creates a back and forth in which the generator tries to create realistic samples to fool the discriminator and the discriminator tries to establish better rules for distinguishing the real from the fake samples. This adversarial relationship is what propels the training of GANs.


<h1><center>

![GAN](GAN.png)

</center></h1>




 One classic example of this is face generation. The faces below are all fake generated using a GAN architecture. The amount of photorealism displays the power of GANs for image generation.



<h1><center>

![Faces](faces.pbm)

</center></h1>



## Style Transfer Methods

One of my favorite topics within generative modeling is style transfer. This is the process of taking images from two different domains $A$ and $B$ and learning a mapping between the two domains. Thus instead of generating random images from domain $B$ we're looking to generate an image in domain $B$ that carries some characteristics of some sample image $x$ in domain $A$. This is known as style transfer because we want to preserve the content of image $x$ while changing the "style" of the image to be like domain $B$. One popular approach to this problem is called CycleGAN, which uses a dual-GAN archiecture that learns a generator $G$ from domain $A$ to $B$ an a separate generator $F$ that maps images from domain $B$ to $A$. Each generator has a corresponding discriminator that learns to predict whether a given image is a real sample from domain $A$ or a fake sample. The model then uses a cycle-consistency loss to ensure that $x \approx F(G(x))$ i.e. the model is able to reconstruct the original sample to ensure that the content is being preserved.


<h1><center>

![cycle](cycle.png)

</center></h1>

Now lets take a look at some example generations from the cycleGAN paper! This first set of examples is trained on a data set of horse images and zebra images, which is one of the benchmark datasets for style transfer.


<h1><center>

![zebra](zebra.png)

</center></h1>

This next example is generated using images from Yosemite National Park taken during Summer and Winter respectively:


<h1><center>

![yosemite](yosemite.png)

</center></h1>

Lastly, this example displays the impressive results of cycleGAN applied to translating real-world photographs into paintings of famous artists and art styles. Each model was trained separately but the results are displayed side-by-side.


<h1><center>

![full](full.png)

</center></h1>


The model also produces impressive results for generating in the opposite direction:


<h1><center>

![Monet](monet.png)

</center></h1>